In [87]:
%pip install -r requirements.txt -U -q

Note: you may need to restart the kernel to use updated packages.


In [88]:
import pandas as pd

# read massaged CSV
df = pd.read_csv('county_records.csv', na_filter=False)
df.head()

,ID,county,burned,published,court_count1,court_count_2,fullName
0,vas_accomack,Accomack,0,2,,1,ACCOMACK
1,vas_albemarle,Albemarle,1,1,,,ALBEMARLE
2,vas_amelia,Amelia,0,2,,1,AMELIA
3,vas_amherst,Amherst,0,,,1,AMHERST
4,vas_augusta,Augusta,0,,,1,AUGUSTA


In [89]:
import geopandas as gpd

counties = gpd.read_file('massaged.geojson')
counties.head()

,fullName,change,dates,geometry
0,GLOUCESTER,[1651] GLOUCESTER created from YORK.,"{'start': '1651-12-31T00:00:00Z', 'end': '1776...","MULTIPOLYGON (((-76.27557 37.30574, -76.27869 ..."
1,FINCASTLE (extinct),FINCASTLE (extinct) created from BOTETOURT.,"{'start': '1772-12-01T00:00:00Z', 'end': '1776...","MULTIPOLYGON (((-89.47247 36.51375, -89.48511 ..."
2,NORFOLK (extinct),NORFOLK (extinct) created from LOWER NORFOLK; ...,"{'start': '1691-05-16T00:00:00Z', 'end': '1776...","MULTIPOLYGON (((-76.37459 36.88549, -76.37656 ..."
3,West Augusta District (extinct),West Augusta District (extinct) created by Vir...,"{'start': '1773-10-11T00:00:00Z', 'end': '1776...","MULTIPOLYGON (((-78.80811 40.72065, -78.86011 ..."
4,BEDFORD,BEDFORD gained from ALBEMARLE and LUNENBURG.,"{'start': '1755-01-01T00:00:00Z', 'end': '1776...","MULTIPOLYGON (((-79.43516 37.60388, -79.42851 ..."


In [90]:
merged = df.merge(counties, on='fullName', how='outer')

# # convert to GeoDataframe
data = gpd.GeoDataFrame(merged)
data.head()

,ID,county,burned,published,court_count1,court_count_2,fullName,change,dates,geometry
0,vas_accomack,Accomack,0,2,,1,ACCOMACK,[1663] ACCOMACK created from NORTHAMPTON.,"{'start': '1663-12-31T00:00:00Z', 'end': '1776...","MULTIPOLYGON (((-75.81141 37.79399, -75.81149 ..."
1,vas_albemarle,Albemarle,1,1,,,ALBEMARLE,"ALBEMARLE gained from LOUISA, lost to creation...","{'start': '1761-05-01T00:00:00Z', 'end': '1776...","MULTIPOLYGON (((-78.58292 38.25014, -78.36980 ..."
2,vas_amelia,Amelia,0,2,,1,AMELIA,AMELIA lost to creation of PRINCE EDWARD.,"{'start': '1754-01-01T00:00:00Z', 'end': '1776...","MULTIPOLYGON (((-77.96820 37.48573, -77.95902 ..."
3,vas_amherst,Amherst,0,,,1,AMHERST,AMHERST created from ALBEMARLE.,"{'start': '1761-05-01T00:00:00Z', 'end': '1776...","MULTIPOLYGON (((-78.83921 38.04757, -78.69671 ..."
4,vas_augusta,Augusta,0,,,1,AUGUSTA,AUGUSTA lost to creation of West Augusta Distr...,"{'start': '1773-10-11T00:00:00Z', 'end': '1776...","MULTIPOLYGON (((-79.92785 39.58906, -79.91059 ..."


In [91]:
# burn_colors = { '': '#00000000', '0': '#08AAD8', '1': '#73D0F2' }
burn_colors = { '': '', '0': '#dddddd', '1': '#cc0024' }
# publish_colors = { '': '#00000000', '0': '#FEF0E3', '1': '#F79768', '2': '#F26B28' }
publish_colors = { '': '#00000000', '0': '#dddddd', '1': '#7bb3d1', '2': '#016eae' }
# april_colors = { '': '#00000000', '0': '#FEF0E3', '1': '#F79768', '2': '#F26B28' }
april_colors = { '': '#00000000', '0': '#F0DCDA', '1': '#E6B8B8', '2': '#DA9695' }
may_colors = { '': '#00000000', '0': '#D4D6DC', '1': '#C3CCD7', '2': '#A0B5D2' }

legend_dict = {
    'Orders Not Burned': 'dddddd',
    'Orders Burned': 'cc0024', 
    'Resolves Not Published': '7bb3d1',
    'Resolves Published': '7bb3d1',
    'Resolves Listed (not printed)': '016eae', 
    'County Court Did Not Meet (April)': 'F0DCDA',
    'County Court  Met (April)': 'E6B8B8',
    'County Court Scheduled, no meeting (April)': 'DA9695',
    'County Court Met (May)': 'D4D6DC',
    'County Court Did Not Meet (May)': 'C3CCD7',
    'County Court Scheduled, no meeting (May)': 'A0B5D2'
}

for row in df.itertuples():
    data.loc[row.Index, 'burn_style'] = burn_colors[row.burned]
    data.loc[row.Index, 'burn_opacity'] = 0.4 if row.burned != '' else 0

    data.loc[row.Index, 'published_style'] = publish_colors[row.published]
    data.loc[row.Index, 'published_opacity'] = 0.4 if row.published != '' else 0

    data.loc[row.Index, 'april_style'] = april_colors[row.court_count1]
    data.loc[row.Index, 'april_opacity'] = 0.4 if row.court_count1 != '' else 0

    data.loc[row.Index, 'may_style'] = may_colors[row.court_count_2]
    data.loc[row.Index, 'may_opacity'] = 0.4 if row.court_count_2 != '' else 0

In [92]:
burned_layer = data.filter(['ID', 'burned', 'geometry', 'fullName', 'burn_style', 'burn_opacity']).copy().rename(columns={'burn_style': "fillColor", 'burn_opacity': 'opacity'})
published_layer = data.filter(['ID', 'published', 'geometry', 'fullName', 'published_style', 'published_opacity']).copy().rename(columns={'published_style': "fillColor", 'published_opacity': 'opacity'})
april_layer = data.filter(['ID', 'court_count1', 'geometry', 'fullName', 'april_style', 'april_opacity']).copy().rename(columns={'april_style': "fillColor", 'april_opacity': 'opacity'})
may_layer = data.filter(['ID', 'court_count_2', 'geometry', 'fullName', 'may_style', 'may_opacity']).copy().rename(columns={'may_style':"fillColor", 'may_opacity': 'opacity'})
# burned_layer.head()

In [93]:
import leafmap.foliumap as leafmap
from ipyleaflet import Map, basemaps

attr = (
    '&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> '
    'contributors, &copy; <a href="https://cartodb.com/attributions">CartoDB</a>'
)
tiles = "https://{s}.basemaps.cartocdn.com/light_nolabels/{z}/{x}/{y}.png"

# calculate centroid to look at
centroid = [
    sum(counties.geometry.total_bounds[[1, 3]]) / 2,
    sum(counties.geometry.total_bounds[[0, 2]]) / 2
]

print(centroid)

m = leafmap.Map(
    basemap=basemaps.Gaode.Satellite,
    draw_control=False,
    measure_control=False,
    center=(centroid),
    zoom=6
)

m.add_tile_layer(
    url=tiles,
    name="light_nolabels",
    attribution=attr
)

[38.6562172417811, -82.4068916037177]


In [94]:
m.add_gdf(data, layer_name="Counties", style={"color": "black"})

callback = lambda feat: { "color": "black", "fillColor": feat["properties"]["fillColor"]}


m.add_gdf(
    burned_layer, 
        layer_name="Orders Burned", 
        style_callback=callback
        # style_callback = lambda x: {
        #     "color": "black",
        #     "fillColor": x['properties']['fillColor'],
        #     "fillOpacity": x['properties']['opacity']
        # }
    )
# m.add_geojson(burned_layer, 'Burned Layers')
# folium.GeoJson(
#     burned_layer,
#     style_function=lambda x: {
#         "fillColor": x['properties']['fillColor'],
#         "color": "black",
#         "fillOpacity": x['properties']['opacity']
#     },
#     popup=burned_popup
# ).add_to(burned_fg)

m.add_layer_control()

In [95]:
m.add_legend(title="Classification", legend_dict=legend_dict)


In [96]:
m

In [97]:
m.save("map.html")

See https://leafmap.org/notebooks/06_legend/

In [98]:
import leafmap

Map = leafmap.Map()

legend_dict = {
    "Burned": "466b9f",
    "Not Burned": "d1def8"
}

# legend_dict = {
#     "11 Open Water": "466b9f",
#     "12 Perennial Ice/Snow": "d1def8",
#     "21 Developed, Open Space": "dec5c5",
#     "22 Developed, Low Intensity": "d99282",
#     "23 Developed, Medium Intensity": "eb0000",
#     "24 Developed High Intensity": "ab0000",
#     "31 Barren Land (Rock/Sand/Clay)": "b3ac9f",
#     "41 Deciduous Forest": "68ab5f",
#     "42 Evergreen Forest": "1c5f2c",
#     "43 Mixed Forest": "b5c58f",
#     "51 Dwarf Scrub": "af963c",
#     "52 Shrub/Scrub": "ccb879",
#     "71 Grassland/Herbaceous": "dfdfc2",
#     "72 Sedge/Herbaceous": "d1d182",
#     "73 Lichens": "a3cc51",
#     "74 Moss": "82ba9e",
#     "81 Pasture/Hay": "dcd939",
#     "82 Cultivated Crops": "ab6c28",
#     "90 Woody Wetlands": "b8d9eb",
#     "95 Emergent Herbaceous Wetlands": "6c9fb8",
# }

Map.add_legend(title="Classification", legend_dict=legend_dict)
Map

Map(center=[20, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…